In [1]:
import os
from smosaic import collection_query, collection_get_data, mosaic

In [2]:
path_dir = os.path.abspath("")

In [3]:
collection=collection_query(
    collection="AMZ1-WFI-L4-SR-1",
    start_date="2025-01-01",
    end_date="2025-02-28",
    bbox="-74.8828,-34.5970,-31.8164,5.5285",
    bands=["BAND3","BAND4", "CMASK"]
)

In [4]:
collection_get_data(collection)

AMAZONIA_1_WFI_20250228_037_01... : 291MB [00:31, 9.24MB/s]                            
AMAZONIA_1_WFI_20250228_037_01... : 318MB [00:33, 9.50MB/s]                            
AMAZONIA_1_WFI_20250228_037_01... : 14.4MB [00:01, 10.8MB/s]                            
AMAZONIA_1_WFI_20250228_037_01... : 281MB [00:27, 10.1MB/s]                            
AMAZONIA_1_WFI_20250228_037_01... : 310MB [00:28, 11.1MB/s]                            
AMAZONIA_1_WFI_20250228_037_01... : 12.6MB [00:01, 11.4MB/s]                            
AMAZONIA_1_WFI_20250228_037_01... : 295MB [00:26, 11.2MB/s]                            
AMAZONIA_1_WFI_20250228_037_01... : 324MB [00:29, 11.0MB/s]                            
AMAZONIA_1_WFI_20250228_037_01... : 10.7MB [00:00, 11.5MB/s]                            
AMAZONIA_1_WFI_20250227_034_01... : 295MB [00:26, 11.3MB/s]                            
AMAZONIA_1_WFI_20250227_034_01... : 319MB [00:39, 7.98MB/s]                            
AMAZONIA_1_WFI_20250227_034_0

In [ ]:
#from json import load

#geojson_data = load(open('br.geojson', 'r', encoding='utf-8'))

In [ ]:
import rasterio

def count_pixels_with_value(raster_path, target_value):
    """
    Counts the number of pixels in a raster that match a specific value.
    
    Args:
        raster_path (str): Path to the raster file
        target_value (int/float): The pixel value to count
        
    Returns:
        int: Count of pixels with the target value
    """
    # Open the raster file
    with rasterio.open(raster_path) as src:
        # Read all data (assuming single-band raster)
        data = src.read(1)
        
        # Count pixels with the target value
        count = (data == target_value).sum()
        
        return dict(total=data.size, count=count)
    
def local_mosaic(data_dir, collection, output_dir, start_year, start_month, start_day, duration_months, bands, mosaic_method, geom=None, grid=None, grid_id=None):
    
    #bdc_grids_data = load_json()
    #selected_tile = ''
    #for g in bdc_grids_data['grids']:
    #    if (g['name'] == grid):
    #        for tile in g['features']:
    #            if tile['properties']['tile'] == grid_id:
    #                selected_tile = tile
    #geometry = selected_tile['properties']['geometry']
    
    if (mosaic_method=='lcf'):
        data_dir = os.path.join(data_dir+'/'+collection)
        for path in os.listdir(data_dir)[:3]:
            pixel_count = count_pixels_with_value(os.path.join(data_dir, path, 'CMASK'), 127)
            print(pixel_count)
            

In [ ]:
result = local_mosaic(
    data_dir=os.path.join(path_dir),
    collection="AMZ1-WFI-L4-SR-1", 
    grid="BDC_LG_V2",
    grid_id="007004",
    output_dir="/mosaic-amazonia-brazil-2m", 
    mosaic_method="lcf", 
    start_year=2025,
    start_month=1,
    start_day=1,
    duration_months=2, 
    bands=['BAND3'],
    #geom=geojson_data['features'][0]['geometry']
)